# Remeshing

This notebook provides examples of triangular mesh adjustment operations, such as:
- decimation - reducing number of triangles
- tesselation - increasing number of triangles

The example uses GPU-accelerated algorithm described in [MicroMap Toolkit](https://github.com/NVIDIAGameWorks/Displacement-MicroMap-Toolkit) repository.

## Initialization

Download sample meshes from AhmedML and DrivAerML datasets.

In [ ]:
%%bash
if [ ! -f "./assets/boundary_1_1M.vtp" ]; then
    mkdir -p ./assets
    wget "https://huggingface.co/datasets/neashton/ahmedml/resolve/main/run_1/boundary_1.vtp" -O "./assets/boundary_1_1M.vtp"
else
    echo "File already exists."
fi

if [ ! -f "./assets/boundary_1_10M.vtp" ]; then
    mkdir -p ./assets
    wget "https://huggingface.co/datasets/neashton/drivaerml/resolve/main/run_1/boundary_1.vtp" -O "./assets/boundary_1_10M.vtp"
else
    echo "File already exists."
fi

Import required packages.

In [1]:
from time import time

import numpy as np

import pyvista as pv

import micromesh_python as pymm

Prepare sample data from the dataset.

In [2]:
# Set to True to use large 10M mesh.
use_10M_mesh = False
filename = "./assets/boundary_1_10M.vtp" if use_10M_mesh else "./assets/boundary_1_1M.vtp"

# Read the mesh from VTP file.
orig_mesh = pv.read(filename)

# The algorithm works only with tri-meshes.
tri_mesh = orig_mesh.triangulate()

In [3]:
# Print some stats.
#
from collections import Counter
from vtk import vtkCellTypes

def get_cell_type_name(cell_type_id: int) -> str:
    name = vtkCellTypes.GetClassNameFromTypeId(cell_type_id)
    return name if name else f"Unknown ({cell_type_id})"

cell_type_counts = {
    get_cell_type_name(k): v
    for k, v in Counter(
        orig_mesh.GetCellType(i) for i in range(orig_mesh.n_cells)
    ).items()
}

print(f"Original mesh:\n{orig_mesh}\nCell type counts:\n{cell_type_counts}\n")
print(f"Tri-mesh:\n{tri_mesh}")

Original mesh:
PolyData (0x7ffb6d3472e0)
  N Cells:    1101574
  N Points:   1131049
  N Strips:   0
  X Bounds:   -1.188e+00, 2.168e-19
  Y Bounds:   -1.847e-01, 1.847e-01
  Z Bounds:   5.000e-02, 3.036e-01
  N Arrays:   5
Cell type counts:
{'vtkQuad': 1101164, 'vtkPolygon': 410}

Tri-mesh:
PolyData (0x7ffb6d347220)
  N Cells:    2203558
  N Points:   1131049
  N Strips:   0
  X Bounds:   -1.188e+00, 2.168e-19
  Y Bounds:   -1.847e-01, 1.847e-01
  Z Bounds:   5.000e-02, 3.036e-01
  N Arrays:   5


Initialize micromesh context.

In [4]:
context = pymm.createContext(verbosity=pymm.Verbosity.Info)

_______________
Vulkan Version:
 - available:  1.3.296
 - requesting: 1.3.0
______________________
Used Instance Layers :

Used Instance Extensions :
____________________
Devices : 3
0: NVIDIA TITAN RTX
  - Compatible 
1: NVIDIA GeForce RTX 3090
  - Compatible 
2: llvmpipe (LLVM 19.1.1, 256 bits)
  - Compatible 
Compatible physical devices found : 3
Using Device:
 - Device Name    : NVIDIA GeForce RTX 3090
 - Vendor         : NVIDIA
 - Driver Version : 535.732.64
 - API Version    : 1.3.242
 - Device Type    : Discrete GPU
________________________
Used Device Extensions :
VK_KHR_acceleration_structure
VK_KHR_ray_tracing_pipeline
VK_KHR_ray_query
VK_KHR_push_descriptor
VK_KHR_deferred_host_operations
VK_KHR_buffer_device_address
VK_EXT_memory_budget
VK_EXT_shader_atomic_float

Context created


## Remeshing

The following sections demonstrate how to use micro_mesh API to perform mesh decimation and tesselation.

### Decimation

Next section defines some helpful utility functions.

In [5]:
def decimate(context, mm_mesh: pymm.Mesh, target_reduction: float = 0.5) -> pymm.Mesh:
    """Decimates source mesh given parameters."""

    remesher_settings = pymm.RemesherSettings()
    remesher_settings.decimationRatio = 1.0 - target_reduction
    remesher_settings.errorThreshold = 90
    remesher_settings.maxSubdivLevel = 4

    return pymm.remesh(context, mm_mesh, remesher_settings)


def create_mm_mesh(vertices: np.ndarray, triangles: np.ndarray) -> pymm.Mesh:
    """Creates micro-mesh Mesh given source mesh as vertices/triangles arrays."""
    assert vertices.ndim == 2 and vertices.shape[1] == 3, f"{vertices.shape = }"
    assert triangles.ndim == 2 and triangles.shape[1] == 3, f"{triangles.shape = }"

    mm_mesh = pymm.Mesh()
    mm_mesh.triangleVertices = triangles
    mm_mesh.vertexPositions = vertices

    return mm_mesh


def mm_mesh_to_polydata(src: pymm.Mesh) -> pv.PolyData:
    """Creates PolyData mesh given source micro-mesh Mesh."""

    # In PolyData, triangular faces are in [3, n0, n1, n2] format.
    num_triangles = np.full((src.triangleVertices.shape[0], 1), 3)
    faces = np.hstack((num_triangles, src.triangleVertices))

    return pv.PolyData(src.vertexPositions, faces)


def polydata_to_mm_mesh(src: pv.PolyData) -> pymm.Mesh:
    """Converts VTK PolyData to micro-mesher Mesh."""

    vertices = src.points
    triangles = np.reshape(src.faces, (-1, 4))
    assert (triangles[:, 0] == 3).all(), "Expected tri-mesh."
    # Remove vertex count, keep only indices.
    triangles = triangles[:, 1:]

    return create_mm_mesh(vertices, triangles)


def warmup_data():
    """Small mesh used to warmup micromesher."""

    # 5 vertices (square with center), 4 triangles.
    vertices = np.array([
        [0, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [1, 1, 0],
        [.5, .5, 0],
    ], dtype=np.float32)

    triangles = np.array([
        [0, 1, 4],
        [1, 3, 4],
        [3, 2, 4],
        [0, 2, 4],
    ], dtype=np.float32)

    return create_mm_mesh(vertices, triangles)

Run mesh decimation.

In [6]:
# Warmup.
decimate(context, warmup_data(), 0.9)

# GPU micromesh decimation.
target_reduction = 0.95

mm_mesh = polydata_to_mm_mesh(tri_mesh)
start = time()
decimated_mesh_gpu = decimate(context, mm_mesh, target_reduction=target_reduction)
elapsed_gpu = time() - start

start = time()
decimated_mesh_cpu = tri_mesh.decimate_pro(target_reduction)
elapsed_cpu = time() - start

  Triangles: 4 -> 4
Remeshing started 2203558 -> 110177 triangles max
Remeshing in progress 2203558 -> 2049292 triangles - 7.3% (511.15 ms)
Remeshing in progress 2203558 -> 1910136 triangles - 13.9% (613.36 ms)
Remeshing in progress 2203558 -> 1777432 triangles - 20.2% (712.81 ms)
Remeshing in progress 2203558 -> 1653954 triangles - 26.0% (809.26 ms)
Remeshing in progress 2203558 -> 1541244 triangles - 31.3% (902.38 ms)
Remeshing in progress 2203558 -> 1343276 triangles - 40.7% (1077.24 ms)
Remeshing in progress 2203558 -> 1176214 triangles - 48.6% (1236.39 ms)
Remeshing in progress 2203558 -> 1033648 triangles - 55.3% (1380.40 ms)
Remeshing in progress 2203558 -> 909780 triangles - 61.2% (1508.84 ms)
Remeshing in progress 2203558 -> 804540 triangles - 66.2% (1623.59 ms)
Remeshing in progress 2203558 -> 682002 triangles - 72.0% (1775.56 ms)
Remeshing in progress 2203558 -> 569878 triangles - 77.3% (1947.51 ms)
Remeshing in progress 2203558 -> 463030 triangles - 82.3% (2161.41 ms)
Remes

Report performance and visualize resulting meshes.

In [7]:
from tabulate import tabulate

headers = ["Device", "Time (sec)", "In faces", "Out faces"]
data = [
    ["GPU", elapsed_gpu, tri_mesh.n_cells, decimated_mesh_gpu.triangleVertices.shape[0]],
    ["CPU", elapsed_cpu, tri_mesh.n_cells, decimated_mesh_cpu.n_cells],
]

print(tabulate(data, headers, tablefmt="github", intfmt=",", floatfmt=".2f"))

| Device   |   Time (sec) |   In faces |   Out faces |
|----------|--------------|------------|-------------|
| GPU      |         4.33 |  2,203,558 |     104,988 |
| CPU      |        11.25 |  2,203,558 |     110,176 |


Display nice side-by-side interactive plot to show the results of the decimation.

In [ ]:
def plot_meshes(mesh_orig: pv.PolyData, mesh_gpu: pymm.Mesh, mesh_cpu: pv.PolyData):
    """Displays meshes."""

    plotter = pv.Plotter(shape=(1, 3))

    plotter.subplot(0, 0)
    plotter.add_mesh(mesh_orig, color="lightgrey")
    plotter.add_text("Mesh", position="upper_left")

    plotter.subplot(0, 1)
    plotter.add_mesh(mm_mesh_to_polydata(mesh_gpu), color="lightblue")
    plotter.add_text("GPU", position="upper_left")

    plotter.subplot(0, 2)
    plotter.add_mesh(mesh_cpu, color="lightgreen")
    plotter.add_text("CPU", position="upper_left")

    plotter.show()

plot_meshes(tri_mesh, decimated_mesh_gpu, decimated_mesh_cpu)

In [8]:
from physicsnemo.sym.geometry.tessellation import Tessellation
# vtp_mesh = pv.read(filename)
# vtp_mesh = vtp_mesh.triangulate()
# vtp_mesh.save("./assets/mesh.stl")
# g = Tessellation.from_stl("./assets/mesh.stl")
decimated_mesh_gpu_pd = mm_mesh_to_polydata(decimated_mesh_gpu)
print(decimated_mesh_gpu_pd)
decimated_mesh_gpu_pd.save("./assets/mesh.stl")
geometry = Tessellation.from_stl("./assets/mesh.stl")

/usr/local/lib/python3.12/dist-packages/warp/codegen.py:2735: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  ast.Str: emit_String,  # Deprecated in 3.8; use Constant
/usr/local/lib/python3.12/dist-packages/warp/codegen.py:2736: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  ast.Num: emit_Num,  # Deprecated in 3.8; use Constant
/usr/local/lib/python3.12/dist-packages/warp/codegen.py:2737: DeprecationWarning: ast.NameConstant is deprecated and will be removed in Python 3.14; use ast.Constant instead
  ast.NameConstant: emit_NameConstant,  # Deprecated in 3.8; use Constant
/usr/local/lib/python3.12/dist-packages/warp/codegen.py:2754: DeprecationWarning: ast.Ellipsis is deprecated and will be removed in Python 3.14; use ast.Constant instead
  ast.Ellipsis: emit_Ellipsis,


PolyData (0x7ffa191eac80)
  N Cells:    104988
  N Points:   52496
  N Strips:   0
  X Bounds:   -1.188e+00, 3.858e-05
  Y Bounds:   -1.847e-01, 1.847e-01
  Z Bounds:   4.994e-02, 3.034e-01
  N Arrays:   0


In [10]:
samples = geometry.sample_boundary(100_000)
tess_cpu = pv.PolyData(np.hstack((samples["x"], samples["y"], samples["z"])))
print(tess_cpu)

Warp 1.6.2 initialized:
   CUDA Toolkit 12.8, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce RTX 3090" (24 GiB, sm_86, mempool enabled)
     "cuda:1"   : "NVIDIA TITAN RTX" (24 GiB, sm_75, mempool enabled)
   CUDA peer access:
     Not supported
   Kernel cache:
     /root/.cache/warp/1.6.2
Module physicsnemo.utils.sdf 36297e8 load on device 'cuda:0' took 0.40 ms  (cached)
PolyData (0x7ffa18961540)
  N Cells:    100000
  N Points:   100000
  N Strips:   0
  X Bounds:   -1.188e+00, 1.737e-05
  Y Bounds:   -1.847e-01, 1.847e-01
  Z Bounds:   4.996e-02, 3.034e-01
  N Arrays:   0


In [ ]:
# from utils import mesh
# from pxr import Usd, UsdGeom

# reefcrab_stage = Usd.Stage.Open('./assets/reefcrab/reefcrab.usd')

# reefcrab_prim = reefcrab_stage.GetPrimAtPath('/World/High/Reefcrab')

# reefcrab_mesh, reefcrab_mesh_xform = mesh.get_mesh(reefcrab_prim)
# print(f'Triangles in low-resolution mesh: {reefcrab_mesh.triangleVertices.shape[0]}')

In [ ]:
# remesher_settings = pymm.RemesherSettings()
# remesher_settings.decimationRatio=0.1
# remesher_settings.errorThreshold=90
# remesher_settings.maxSubdivLevel=4

# reefcrab_mesh.vertexNormals = np.empty(0, dtype=np.float32)
# remeshed_mesh = pymm.remesh(context, reefcrab_mesh, remesher_settings)

# print(f'Triangles in remeshed mesh: {remeshed_mesh.triangleVertices.shape[0]}')
# print(f'Actual decimation ratio: {remeshed_mesh.triangleVertices.shape[0] / reefcrab_mesh.triangleVertices.shape[0]:.2f}')

In [ ]:
from utils import mesh
from pxr import Usd, UsdGeom

stage = Usd.Stage.Open('./assets/simplewall/simplewall.usd')

for prim in stage.Traverse():
    if prim.IsA(UsdGeom.Mesh):
        mesh_prim = UsdGeom.Mesh(prim)
        print(f'{mesh_prim.GetPrim().GetPrimPath()}: vertices={len(mesh_prim.GetPointsAttr().Get())}, faces={int(len(mesh_prim.GetFaceVertexIndicesAttr().Get())/3)}')

low_prim = stage.GetPrimAtPath('/World/Low/Simplewall')
high_prim = stage.GetPrimAtPath('/World/High/Simplewall')

# Get the low and high resolution primitives from the stage and convert to the necessary format:
low_mesh, low_mesh_xform = mesh.get_mesh(low_prim)
high_mesh, high_mesh_xform = mesh.get_mesh(high_prim)

In [11]:
pretessellator_settings = pymm.PreTessellatorSettings()
pretessellator_settings.maxSubdivLevel = 4
pretessellator_settings.edgeLengthBased = True

# low_mesh = decimated_mesh_gpu
# pretessellated_mesh = pymm.preTessellate(context, low_mesh, pretessellator_settings)


# pretessellator_settings = pymm.PreTessellatorSettings()
# pretessellator_settings.maxSubdivLevel = 4
# pretessellator_settings.edgeLengthBased = True

# low_mesh = reefcrab_mesh

low_mesh = pymm.Mesh()
low_mesh.triangleVertices = decimated_mesh_gpu.triangleVertices
low_mesh.vertexPositions = decimated_mesh_gpu.vertexPositions

pretessellated_mesh = pymm.preTessellate(context, low_mesh, pretessellator_settings)
print(f'Triangles in low-resolution mesh: {low_mesh.triangleVertices.shape[0]}')
print(f'Triangles in remeshed mesh: {pretessellated_mesh.triangleVertices.shape[0]}')

Generating mesh attributes TriangleSubdivLevels|TrianglePrimitiveFlags|VertexNormals|VertexDirections
  Triangles: 1908212 -> 1908212
Triangles in low-resolution mesh: 104988
Triangles in remeshed mesh: 1908212


In [ ]:
plot_meshes(mm_mesh_to_polydata(decimated_mesh_gpu), pretessellated_mesh, tess_cpu)